In [92]:
import pandas as pd
import plotly.graph_objects as go

2 fuentes:
- Plazos fijos: https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/pas2023.xls
- CER: https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/cer2023.xls

In [93]:
def read_pf_trad_bcos(anio:int = 2023):
    df = pd.read_excel(f"https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/pas{anio}.xls", sheet_name="Estra_dia_bancos", header=25,index_col="fecha")
    df = df[["pasdes18","pasdes20","pasdes22","pasdes24"]] / 1_000 #Está en miles, lo paso a millones de pesos
    df = df.rename({"pasdes18":"pf_trad_99.000", "pasdes20":"pf_trad_100.000 a 499.999",
                                    "pasdes22":"pf_trad_500.000 a 999.999", "pasdes24":"pf_trad_1.000.000+"},axis=1)
    return df

pf_trad_df = []
for anio in [2020,2021,2022,2023]:
    temp = read_pf_trad_bcos(anio=anio)
    pf_trad_df.append(temp)
pf_trad_df = pd.concat(pf_trad_df)
pf_trad_df

,pf_trad_99.000,pf_trad_100.000 a 499.999,pf_trad_500.000 a 999.999,pf_trad_1.000.000+
fecha,,,,
20200102,3271.352,15352.223,10462.915,76408.098
20200103,3397.743,16271.182,11528.363,51287.046
20200106,2816.056,13176.455,9026.834,48506.163
20200107,1613.064,7278.187,4715.954,24213.779
20200108,2854.890,14360.960,10075.839,58416.095
...,...,...,...,...
20231221,946.976,13573.400,18304.608,426550.045
20231222,1315.352,19507.557,29477.376,566373.760
20231226,2100.797,26033.733,37496.987,653824.194


In [94]:
def read_pf_uva(anio:int = 2023):
    df = pd.read_excel(f"https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/pas{anio}.xls", sheet_name="UVA_UVI", header=25, index_col="fecha")
    df = df[["pasvar01","pasvar04","pasvar07","pasvar10"]] / 1_000 #Está en miles, lo paso a millones de pesos
    df = df.rename({"pasvar01":"pf_uva_99.000", "pasvar04":"pf_uva_100.000 a 499.999",
                                    "pasvar07":"pf_uva_500.000 a 999.999", "pasvar10":"pf_uva_1.000.000+"},axis=1)
    return df

pf_uva_df = []
for anio in [2020,2021,2022,2023]:
    temp = read_pf_uva(anio=anio)
    pf_uva_df.append(temp)
pf_uva_df = pd.concat(pf_uva_df)
pf_uva_df

,pf_uva_99.000,pf_uva_100.000 a 499.999,pf_uva_500.000 a 999.999,pf_uva_1.000.000+
fecha,,,,
20200102,21.202,51.253,35.153,786.613
20200103,15.057,39.073,21.533,187.447
20200106,19.082,47.981,25.533,2007.373
20200107,14.209,31.178,17.425,191.170
20200108,15.062,40.351,23.439,404.594
...,...,...,...,...
20231221,54.356,706.417,915.731,70072.971
20231222,60.889,867.342,1267.782,3494.024
20231226,68.700,888.341,1208.168,4795.534


In [97]:
def read_cer(anio:int = 2023):
    df = pd.read_excel(f"https://www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/cer{anio}.xls", header=25, index_col="fecha")
    df = df[["coef001"]]
    df = df.rename({"coef001":"cer"},axis=1)
    return df

cer_df = []
for anio in [2020,2021,2022,2023]:
    temp = read_cer(anio=anio)
    cer_df.append(temp)
cer_df = pd.concat(cer_df)
cer_df = cer_df.drop_duplicates()
cer_df

,cer
fecha,
20200101,18.726329
20200102,18.751778
20200103,18.777262
20200104,18.802781
20200105,18.828335
...,...
20240111,193.008313
20240112,193.759678
20240113,194.513969


In [98]:
df = pf_trad_df.merge(right=pf_uva_df, left_index=True, right_index=True, how="left")
df = df.merge(right=cer_df, left_index=True, right_index=True, how="left")
df.index = pd.to_datetime(df.index, format="%Y%m%d")
df['pf_uva_total'] = df.filter(like='uva').sum(axis=1)
df['pf_trad_total'] = df.filter(like='trad').sum(axis=1)
df

,pf_trad_99.000,pf_trad_100.000 a 499.999,pf_trad_500.000 a 999.999,pf_trad_1.000.000+,pf_uva_99.000,pf_uva_100.000 a 499.999,pf_uva_500.000 a 999.999,pf_uva_1.000.000+,cer,pf_uva_total,pf_trad_total
fecha,,,,,,,,,,,
2020-01-02,3271.352,15352.223,10462.915,76408.098,21.202,51.253,35.153,786.613,18.751778,894.221,105494.588
2020-01-03,3397.743,16271.182,11528.363,51287.046,15.057,39.073,21.533,187.447,18.777262,263.110,82484.334
2020-01-06,2816.056,13176.455,9026.834,48506.163,19.082,47.981,25.533,2007.373,18.853923,2099.969,73525.508
2020-01-07,1613.064,7278.187,4715.954,24213.779,14.209,31.178,17.425,191.170,18.879546,253.982,37820.984
2020-01-08,2854.890,14360.960,10075.839,58416.095,15.062,40.351,23.439,404.594,18.905203,483.446,85707.784
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21,946.976,13573.400,18304.608,426550.045,54.356,706.417,915.731,70072.971,177.885610,71749.475,459375.029
2023-12-22,1315.352,19507.557,29477.376,566373.760,60.889,867.342,1267.782,3494.024,178.578104,5690.037,616674.045
2023-12-26,2100.797,26033.733,37496.987,653824.194,68.700,888.341,1208.168,4795.534,181.375144,6960.743,719455.711


In [99]:
df_ajustado = df.copy()
last_cer = df.cer.iloc[-1]
df_ajustado = df_ajustado.div(df["cer"],axis=0).mul(last_cer).drop("cer",axis=1)
df_ajustado.head()

,pf_trad_99.000,pf_trad_100.000 a 499.999,pf_trad_500.000 a 999.999,pf_trad_1.000.000+,pf_uva_99.000,pf_uva_100.000 a 499.999,pf_uva_500.000 a 999.999,pf_uva_1.000.000+,pf_uva_total,pf_trad_total
fecha,,,,,,,,,,
2020-01-02,31888.740788,149651.599634,101991.220853,744816.831456,206.674513,499.608000,342.667162,7667.807701,8716.757376,1.028348e+06
2020-01-03,33075.833971,158394.238276,112224.562171,499261.368019,146.574603,380.361923,209.616187,1824.730667,2561.283380,8.029560e+05
2020-01-06,27301.852408,127746.617849,87515.763032,470270.514191,185.001274,465.179024,247.544153,19461.616308,20359.340759,7.128347e+05
2020-01-07,15617.539393,70466.746626,45659.439036,234435.612833,137.570250,301.862569,168.707270,1850.890607,2459.030696,3.661793e+05
2020-01-08,27603.272589,138852.808173,97420.962098,564811.741919,145.631002,390.144507,226.626282,3911.926018,4674.327810,8.286888e+05


In [100]:
df_ajustado_mensual = df_ajustado.copy()
df_ajustado_mensual = df_ajustado_mensual.resample('M').sum()
df_ajustado_mensual.index = df_ajustado_mensual.index.month_name(locale='es_ES.utf8').str.lower().str[:3] + "-" + df_ajustado_mensual.index.year.astype(str).str[2:]
df_ajustado_mensual.head()

,pf_trad_99.000,pf_trad_100.000 a 499.999,pf_trad_500.000 a 999.999,pf_trad_1.000.000+,pf_uva_99.000,pf_uva_100.000 a 499.999,pf_uva_500.000 a 999.999,pf_uva_1.000.000+,pf_uva_total,pf_trad_total
fecha,,,,,,,,,,
ene-20,430115.360363,2.094265e+06,1.464514e+06,7.895510e+06,4480.054269,13168.113854,7536.730224,227965.446407,253150.344754,1.188440e+07
feb-20,354308.569698,1.841421e+06,1.308731e+06,7.571273e+06,6084.581279,21483.781992,13636.144091,131254.626100,172459.133462,1.107573e+07
mar-20,322308.371857,1.686350e+06,1.213942e+06,7.113329e+06,3966.824026,12073.510996,7249.503360,95195.576146,118485.414527,1.033593e+07
abr-20,327293.273239,1.792317e+06,1.392819e+06,8.005601e+06,5299.394738,17907.061634,11817.192312,104279.460262,139303.108946,1.151803e+07
may-20,315544.659149,1.688960e+06,1.290570e+06,8.398129e+06,5738.269874,21204.451318,13711.575227,103773.097919,144427.394338,1.169320e+07


In [238]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_trad_total"]/df_ajustado_mensual["pf_trad_total"].iloc[0] * 100, name = "PF tradicional"))
fig.add_trace(go.Scatter(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_uva_total"]/df_ajustado_mensual["pf_uva_total"].iloc[0] * 100, name = "PF UVA"))

fig.update_layout(template = None, separators = ",.", font_family = "georgia",
                  title_text = "Evolución comparada plazos fijos tradicionales vs UVA<br><sup> Ajustados por CER",
                  height = 600, width = 700, legend_orientation = "h", legend_y = 1.08)
fig.update_yaxes(tickformat = ",")
fig.update_xaxes(nticks = 15)
fig.add_vline(x = "jul-22", line_dash = "dash", line_width=.8)
fig.add_annotation(text="Renuncia Guzmán", x="jul-22",showarrow=False, y =170)
fig.add_vline(x="ago-23", line_dash = "dash", line_width = .8)
fig.add_annotation(text="Hiperlicuación", x="nov-23",showarrow=False, y =175)
fig.add_hline(y=100, line_dash = "dash",line_width = .8, annotation_text = "Base Enero 2020 = 100", annotation_position = "top left")
fig.add_annotation(text="Fuente BCRA", showarrow=False, xref="paper",yref="paper", x=0,y=-.09)

In [243]:
fig = go.Figure()
# fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_uva_total"]))
fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_trad_total"]))
ultimo_dato = df_ajustado_mensual["pf_trad_total"].iloc[-1]
ultimo_mes = df_ajustado_mensual.index[-1]
fig.update_layout(template = None, separators = ",.", font_family = "georgia",
                  title_text = f"Plazos fijos tradicional en millones de pesos ajustados por CER<br><sup>Último dato a {ultimo_mes}",
                  height = 600, width = 700, )
fig.update_yaxes(tickformat = ",")
fig.update_xaxes(nticks = 15)
fig.add_vline(x="ago-23", line_dash = "dash", line_width=.7)
fig.add_annotation(text = f"<b>{format(int(ultimo_dato), ',').replace(',', '.')}", x = ultimo_mes, y = ultimo_dato,ax=25,ay=-25)
fig.add_annotation(text="Fuente BCRA", showarrow=False, xref="paper",yref="paper", x=0,y=-.09)

In [247]:
fig = go.Figure()
fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_uva_total"]))
# fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_trad_total"]))
ultimo_dato = df_ajustado_mensual["pf_uva_total"].iloc[-1]
ultimo_mes = df_ajustado_mensual.index[-1]
fig.update_layout(template = None, separators = ",.", font_family = "georgia",
                  title_text = f"Plazos fijos UVA en millones de pesos ajustados por CER<br><sup>Último dato a {ultimo_mes}",
                  height = 600, width = 700, )
fig.update_yaxes(tickformat = ",")
fig.update_xaxes(nticks = 15)
fig.add_vline(x="ago-23", line_dash = "dash", line_width=.7)
fig.add_annotation(text = format(int(ultimo_dato), ',').replace(',', '.'), x = ultimo_mes, y = ultimo_dato,ax=0,ay=-25)
fig.add_annotation(text="Fuente BCRA", showarrow=False, xref="paper",yref="paper", x=0,y=-.09)

In [245]:
fig = go.Figure()

fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_trad_total"], yaxis='y1',offsetgroup=1, name="PF tradicional (izq)"))
fig.add_trace(go.Bar(x = df_ajustado_mensual.index, y = df_ajustado_mensual["pf_uva_total"], yaxis='y2',offsetgroup=2, name = "PF UVA (der)"))
ultimo_dato = df_ajustado_mensual["pf_uva_total"].iloc[-1]
ultimo_mes = df_ajustado_mensual.index[-1]
fig.update_layout(
  template = None, legend_orientation = "h", legend_y=1.05, separators = ",.",
  height = 600, width = 1_000, font_family="georgia",
  title_text = f"Evolución Plazos fijos (UVA vs tradicional) ajustado por CER a precios de hoy<br><sup>En millones de pesos. Último dato a {ultimo_mes}",
  yaxis2=dict(
      overlaying='y',
      side='right',
      tickformat = ","
  )
)
fig.update_yaxes(tickformat = ",")
fig.update_xaxes(nticks = 15)
fig.add_vline(x = "jul-22", line_dash = "dash", line_width=.8)
fig.add_annotation(text="<b>Renuncia Guzmán", x="jul-22",showarrow=False, y = 21_000_000)
fig.add_vline(x="nov-23", line_dash = "dash", line_width = .8)
fig.add_annotation(text="<b>Hiperlicuación", x="nov-23",showarrow=False, y =21_000_000)
fig.add_annotation(text = "Fuente BCRA", xref="paper",yref="paper", x=0, y=-.1, showarrow=False)